
## Informações Gerais
| Informações     | Detalhes                         |
|-----------------|----------------------------------|
|Origem           | silver.brewery                   |
|Nome da Tabela   | gold.breweries_by_city_type      |
|Nome da Tabela   | gold.breweries_by_type           |
|Nome da Tabela   | gold.breweries_by_state_city     |
|Nome da Tabela   | gold.breweries_by_country_state  |
|Nome da Tabela   | gold.breweries_by_city           |

## Histórico de Atualizações
| Data       | Desenvolvido por | Motivo               |
|:----------:|------------------|----------------------|
|19/04/2024  | Gabriel Alves     | Criação do notebook  |


In [0]:
from pyspark.sql.functions import *
import datetime

In [0]:
# Função para salvar DataFrames como tabelas Delta
def save_table_as_delta(df, table_name, path):
    df.write \
        .format("delta") \
        .mode("overwrite") \
        .option("overwriteSchema", "true") \
        .option("path", path) \
        .saveAsTable(f'{schema_gold}.{table_name}')

In [0]:
# Definição de variáveis de configuração
storage_account    = "stbrewerydatalake"       # sua Storage Account
container_name     = "brewery-project"         # container ADLS Gen2
gold_container     = "3.gold"                  # pasta da camada Gold

schema_silver      = "silver"                  # nome do banco de dados no metastore para a Silver
table_silver       = "brewery"                 # nome da tabela Delta da camada Silver
schema_gold        = "gold"                    # nome do banco de dados no metastore para a Gold

# Caminho da camada Gold
output_path = f"abfss://{container_name}@{storage_account}.dfs.core.windows.net/{gold_container}/"

In [0]:
# Criar database Gold se não existir
spark.sql(f"CREATE DATABASE IF NOT EXISTS {schema_gold}")

# 1. Breweries por tipo e cidade
df_brew_by_city_type = spark.sql(f"""
    SELECT
        brewery_type,
        city,
        COUNT(*) AS qtd_breweries
    FROM {schema_silver}.{table_silver}
    GROUP BY brewery_type, city
    ORDER BY qtd_breweries DESC
""")
save_table_as_delta(df_brew_by_city_type, "gold_breweries_by_city_type", f"{output_path}breweries_by_city_type")

# 2. Breweries por tipo
df_brew_by_type = spark.sql(f"""
    SELECT
        brewery_type,
        COUNT(*) AS qtd_breweries
    FROM {schema_silver}.{table_silver}
    GROUP BY brewery_type
    ORDER BY qtd_breweries DESC
""")
save_table_as_delta(df_brew_by_type, "gold_breweries_by_type", f"{output_path}breweries_by_type")

# 3. Breweries por estado e cidade
df_brew_by_state_city = spark.sql(f"""
    SELECT
        state,
        city,
        COUNT(*) AS qtd_breweries
    FROM {schema_silver}.{table_silver}
    GROUP BY state, city
    ORDER BY state, qtd_breweries DESC
""")
save_table_as_delta(df_brew_by_state_city, "gold_breweries_by_state_city", f"{output_path}breweries_by_state_city")

# 4. Breweries por país e estado
df_brew_by_country_state = spark.sql(f"""
    SELECT
        country,
        state,
        COUNT(*) AS qtd_breweries
    FROM {schema_silver}.{table_silver}
    GROUP BY country, state
    ORDER BY country, qtd_breweries DESC
""")
save_table_as_delta(df_brew_by_country_state, "gold_breweries_by_country_state", f"{output_path}breweries_by_country_state")

# 5. Breweries por cidade
df_brew_by_city = spark.sql(f"""
    SELECT
        city,
        COUNT(*) AS qtd_breweries
    FROM {schema_silver}.{table_silver}
    GROUP BY city
    ORDER BY qtd_breweries DESC
""")
save_table_as_delta(df_brew_by_city, "gold_breweries_by_city", f"{output_path}breweries_by_city")


In [0]:
# Após salvar as tabelas Gold
tables_gold = [
    "gold_breweries_by_city_type",
    "gold_breweries_by_type",
    "gold_breweries_by_state_city",
    "gold_breweries_by_country_state",
    "gold_breweries_by_city"
]

# Otimiza todas as tabelas Gold
for table in tables_gold:
    spark.sql(f"OPTIMIZE {schema_gold}.{table}")